In [1]:
# File Authorship Information
__author__ = """Matteo Lulli, Antonino Marcianò, Emanuele Zappalà"""
__copyright__ = """Copyright 2023, Matteo Lulli, Antonino Marcianò, Emanuele Zappalà, idea.deploy"""
__license__ = """Permission is hereby granted, free of charge, 
to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software 
without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, 
and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:
The above copyright notice and this permission notice shall be 
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, 
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES 
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT 
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER 
DEALINGS IN THE SOFTWARE."""
__maintainer__ = "Matteo Lulli, Emanuele Zappalà"
__email__ = "matteo.lulli@gmail.com"
__status__ = "Development"

In [2]:
# Development cell
%load_ext autoreload
%autoreload 2

# Exact Evaluation of Hexagonal Spin-networks and Topological Quantum Neural Networks

Authors: Matteo Lulli (1), Antonino Marcianò (2, 3, 4), Emanuele Zappalà (5)

(1) Department of Mechanics and Aerospace Engineering, Southern University of Science and Technology, Shenzhen, Guangdong 518055, China, [lulli@sustech.edu.cn](mailto:lulli@sustech.edu.cn)

(2) Department of Physics, Fudan University, 200433 Shanghai, China, [marciano@fudan.edu.cn](mailto:marciano@fudan.edu.cn)

(3) Laboratori Nazionali di Frascati INFN, Frascati (Rome), Italy, EU

(4) INFN sezione Roma Tor Vergata, I-00133 Rome, Italy, EU

(5) Department of Mathematics and Statistics, Idaho State University Physical Science Complex |  921 S. 8th Ave., Stop 8085 | Pocatello, ID 83209, USA, [emanuelezappala@isu.edu](mailto:emanuelezappala@isu.edu)

**Abstract:**
The physical scalar product between spin-networks has been shown to be a fundamental tool in the theory of topological quantum neural networks (TQNN), which are quantum neural networks previously introduced by the authors in the context of quantum machine learning. However, the effective evaluation of the scalar product remains a bottleneck for the applicability of the theory.
We introduce an algorithm for the evaluation of the physical scalar product defined by Noui and Perez between spin-network with hexagonal shape. By means of recoupling theory and the properties of the Haar integration we obtain an efficient algorithm, and provide several proofs regarding the main steps. We investigate the behavior of the TQNN evaluations on certain classes of spin-networks with the classical and quantum recoupling. All results can be independently reproduced through the ``idea.deploy" framework [https://github.com/lullimat/idea.deploy](https://github.com/lullimat/idea.deploy)

# Reproducibility

his document is intended for those interested readers who want to reproduce the results reported in the paper [https://arxiv.org/abs/2310.03632](https://arxiv.org/abs/2310.03632). In the present case the computational resources needed should be available in general. 

Next development steps will include a class to measure the time required by each cell and output it in a .json file which can be sent to [matteo.lulli@gmail.com](mailto:matteo.lulli@gmail.com) and [emanuelezappala@isu.edu](mailto:emanuelezappala@isu.edu) so that average execution times will be available and organized according to the hardware.

Each subsection can be executed independently and reproduce the results which will be stored locally in the directory 'reproduced-data', so that the data will be generated only once.

Plots can also be generated using the same scripts employed for the figures of the paper. Since there are some issues in executing these scripts in the Jupyter environment we include them as separated files which are called from the cells themselves. **In order to reproduce the plots a working 'latex' installation is necessary to be present on the system.**

This file will be kept updated for new local features and developments in the parent project [**idea.deploy**](https://github.com/lullimat/idea.deploy)

This file is supposed to be pulled from the repository [https://arxiv.org/abs/2310.03632](https://arxiv.org/abs/2310.03632), from within the "papers" directory the idea.deploy project.